In [1]:
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

from util.metric import MultiScaleSSIM, psnr
from pytorch_msssim import ssim, ms_ssim

In [2]:
# JPEG 파일들이 있는 폴더 경로
jpeg_folder = './data/kodim/jpeg'

# 원본 PNG 파일이 있는 폴더 경로 (참조용)
png_folder = './data/kodim/png'

In [3]:
sampling_factors = ['4:4:4', '4:2:2', '4:2:0']
quality_values = range(10, 101, 10)

png_images_path = os.listdir(png_folder)
png_images_path.sort()


In [4]:
png_images = []
jpeg_images_dict = {sampling: {} for sampling in sampling_factors}
results_dict = {sampling: [] for sampling in sampling_factors}

# PNG 폴더의 모든 파일에 대해 metric 계산
for file_name in png_images_path:
    if file_name.lower().endswith('.png'):
        png_path = os.path.join(png_folder, file_name)
        with Image.open(png_path) as img:
            png_images.append(np.array(img).transpose(2, 0, 1))

for sampling in sampling_factors:
    jpeg_images_quality_dict = {quality: [] for quality in quality_values}
    jpeg_images = []
    for quality in quality_values:
        for i in range(len(png_images)):
            file_name = f"kodim{i+1:02}_{sampling}_{quality}.jpeg"
            # JPEG 파일 이름과 경로 설정
            jpeg_path = os.path.join(jpeg_folder, file_name)
            jpeg_size = os.path.getsize(jpeg_path) * 8
            
            with Image.open(jpeg_path) as img:
                jpeg_img = np.array(img).transpose(2, 0, 1)
                bpp = jpeg_size / (jpeg_img.shape[1] * jpeg_img.shape[2])
                jpeg_images_quality_dict[quality].append([jpeg_img, bpp])
                
        jpeg_images_dict[sampling] = jpeg_images_quality_dict
                        

In [5]:
# MS-SSIM 계산
for sampling in sampling_factors:
    print('[processing] sampling:', sampling)
    for quality in quality_values:
        print('\t[processing] quality:', quality, end='\t | ')
        jpeg_images = jpeg_images_dict[sampling][quality]
        ms_ssim_losses1 = []
        ssim_losses = []
        psnr_losses = []
        for i in range(len(png_images)):
            image1 = np.expand_dims(png_images[i], axis=0)
            image2 = np.expand_dims(jpeg_images[i][0], axis=0)
            ms_ssim_losses1.append(MultiScaleSSIM(image1, image2, max_val=255))
            psnr_losses.append(psnr(image1, image2))
            bpp = jpeg_images[i][1]
        ms_ssim_loss_1 = sum(ms_ssim_losses1) / len(ms_ssim_losses1)
        psnr_loss = sum(psnr_losses) / len(psnr_losses)
        ssim_loss = ssim(torch.tensor(image1).float(), torch.tensor(image2).float(), data_range=255, size_average=True).item()
        ms_ssim_loss_2 = ms_ssim(torch.tensor(image1).float(), torch.tensor(image2).float(), data_range=255, size_average=True).item()
        results_dict[sampling].append((quality, bpp, ssim_loss, ms_ssim_loss_1, ms_ssim_loss_2, psnr_loss))
        print(f'MS-SSIM1: {ms_ssim_loss_1:.4f} | MS-SSIM2: {ms_ssim_loss_2:.4f} | SSIM: {ssim_loss:.4f} | PSNR: {psnr_loss:.4f}')


[processing] sampling: 4:4:4
	[processing] quality: 10	 | MS-SSIM1: 0.9608 | MS-SSIM2: 0.9042 | SSIM: 0.7252 | PSNR: 30.7997
	[processing] quality: 20	 | MS-SSIM1: 0.9788 | MS-SSIM2: 0.9574 | SSIM: 0.8187 | PSNR: 32.2860
	[processing] quality: 30	 | MS-SSIM1: 0.9842 | MS-SSIM2: 0.9718 | SSIM: 0.8595 | PSNR: 33.0513
	[processing] quality: 40	 | MS-SSIM1: 0.9874 | MS-SSIM2: 0.9797 | SSIM: 0.8830 | PSNR: 33.6280
	[processing] quality: 50	 | MS-SSIM1: 0.9894 | MS-SSIM2: 0.9839 | SSIM: 0.8994 | PSNR: 34.1275
	[processing] quality: 60	 | MS-SSIM1: 0.9909 | MS-SSIM2: 0.9871 | SSIM: 0.9137 | PSNR: 34.5953
	[processing] quality: 70	 | MS-SSIM1: 0.9928 | MS-SSIM2: 0.9905 | SSIM: 0.9300 | PSNR: 35.3028
	[processing] quality: 80	 | MS-SSIM1: 0.9948 | MS-SSIM2: 0.9938 | SSIM: 0.9488 | PSNR: 36.4840
	[processing] quality: 90	 | MS-SSIM1: 0.9972 | MS-SSIM2: 0.9969 | SSIM: 0.9718 | PSNR: 39.2067
	[processing] quality: 100	 | MS-SSIM1: 0.9997 | MS-SSIM2: 0.9996 | SSIM: 0.9974 | PSNR: 50.4960
[processin

In [6]:
for sampling in sampling_factors:
    results = results_dict[sampling]
    
    # 결과를 bpp 기준으로 정렬
    results.sort()
    
    # DataFrame 생성
    df = pd.DataFrame(results, columns=['quality', 'bpp', 'SSIM', 'MS-SSIM1', 'MS-SSIM2', 'PSNR'])
    
    # 엑셀 파일로 저장
    excel_path = f'/home/vision/models/AdvancedDigitalSignalProcessingCourse-FinalTermProject/data/kodim/results_{sampling}.xlsx'
    df.to_excel(excel_path, index=False)
    
    print(f'{sampling} 샘플링 팩터에 대한 결과가 {excel_path}에 저장되었습니다.')

4:4:4 샘플링 팩터에 대한 결과가 /home/vision/models/AdvancedDigitalSignalProcessingCourse-FinalTermProject/data/kodim/results_4:4:4.xlsx에 저장되었습니다.
4:2:2 샘플링 팩터에 대한 결과가 /home/vision/models/AdvancedDigitalSignalProcessingCourse-FinalTermProject/data/kodim/results_4:2:2.xlsx에 저장되었습니다.
4:2:0 샘플링 팩터에 대한 결과가 /home/vision/models/AdvancedDigitalSignalProcessingCourse-FinalTermProject/data/kodim/results_4:2:0.xlsx에 저장되었습니다.
